### Imports and configuration

In [107]:
from pathlib import Path
import pandas as pd
import re
import warnings

# Suppress openpyxl warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

# Specify subdirectory to read from (e.g., "01-2026" for January 2026)
month_folder = "02-2026"

# Find all Excel files in the specified subdirectory
daily_report_path = Path("daily-report") / month_folder
excel_files = [f for f in daily_report_path.glob("*.xlsx") 
               if not f.name.startswith("~$")]

In [108]:
# Process each file
dfs = []
for file_path in excel_files:
    # Extract date from filename pattern: "tanggal DD MMM YYYY"
    match = re.search(r"tanggal (\d{1,2}) (\w{3}) (\d{4})", file_path.name)
    
    if match:
        day, month, year = match.groups()
        # Handle both "1 Feb" and "01 Feb" sheet names
        day_no_zero = str(int(day))     # e.g., '1'
        day_zero = day.zfill(2)         # e.g., '01'
        possible_sheet_names = [f"{day_no_zero} {month}", f"{day_zero} {month}"]
        report_date = pd.to_datetime(
            f"{day} {month} {year}", format="%d %b %Y"
        ).date()

        # Try opening each possible sheet name until one works
        df = None
        for sheet in possible_sheet_names:
            try:
                df = pd.read_excel(file_path, sheet_name=sheet, header=13)
                break
            except Exception:
                continue
        if df is not None:
            df["Report Date"] = report_date
            df["Operation Date"] = report_date - pd.Timedelta(days=1)
            dfs.append(df)
        else:
            print(f"Warning: Could not find valid sheet name in {file_path.name}")
    else:
        print(f"Warning: Could not extract date from filename: {file_path.name}")

# Merge all dataframes
if dfs:
    df = pd.concat(dfs, ignore_index=True)
else:
    df = pd.DataFrame()
    print("No dataframes were successfully loaded.")

### Select and filter columns

In [109]:
# Keep only needed columns
selected_columns = ["Zona", "Nama Sumur", "RIG", "Jenis Kegiatan", "Kegiatan ", "Report Date", "Operation Date"]
df = df[selected_columns]

In [110]:
# Filter rows where Zona is in specified values and not NA
df = df[df['Zona'].isin(['Zona 1', 'Zona 2 & 3', 'Zona 4']) & df['Zona'].notna()]

# Rename Zona values for consistency
df['Zona'] = df['Zona'].replace({'Zona 1': 'Zone 1', 'Zona 2 & 3': 'Zone 2&3', 'Zona 4': 'Zone 4'})

# Rename columns for clarity and consistency
df = df.rename(columns={
    'Zona': 'Zone',
    'Nama Sumur': 'Well Name',
    'RIG': 'Rig Name',
    'Jenis Kegiatan': 'Well Type',
    'Kegiatan ': 'Summary Report'
})

### Add reference columns and defaults

In [111]:
# Reference columns from the image
reference_columns = [
    "Flag", "Region", "Zone", "APH", "Rig Name", "Well Name", "Well Name [2]",
    "Well Type", "Location", "Spud Date", "Release Date", "Status",
    "Status Code [1]", "Status Code [2]", "Summary Report", "Current Status",
    "Next Plan", "Report Date", "Operation Date"
]

# Add any missing columns as blank
for col in reference_columns:
    if col not in df.columns:
        df[col] = ""  # blank

# Reorder according to the reference
df = df[reference_columns]

# Set default values for Flag and Region columns
df['Flag'] = "INC"
df['Region'] = "Region 1"
df["Location"] = "Onshore"

In [112]:
# Fill APH column based on Zone values
df["APH"] = df["Zone"].map({"Zone 1": "PEP", "Zone 4": "PEP", "Zone 2&3": "PHR"})

# Replace "Eksplorasi" with "Exploration" in Well Type column
df["Well Type"] = df["Well Type"].replace("Eksplorasi", "Exploration")

### Zone 1

In [113]:
df_z1 = df[df["Zone"] == "Zone 1"].copy()

# Split Well Name on first "/"
well_name_split = df_z1["Well Name"].str.split("/", n=1, expand=True)
df_z1["Well Name"] = well_name_split[0].str.strip()
df_z1["Well Name [2]"] = well_name_split.get(1, pd.Series([""] * len(df_z1))).fillna("").str.strip()

In [114]:
# Split Summary Report into Summary and Plan (case-insensitive "Plan:" or "Plan :")
summary_split = df_z1["Summary Report"].str.split(r"(?i)Plan\s*:\s*", n=1, expand=True, regex=True)
df_z1["Summary Report"] = summary_split[0].fillna("").str.strip()
df_z1["Next Plan"] = summary_split.get(1, pd.Series([""] * len(df_z1))).fillna("").str.strip()

In [115]:
# Remove "Rig" and extra spaces from "Rig Name" column
df_z1["Rig Name"] = df_z1["Rig Name"].str.replace("Rig", "", regex=False).str.strip()

In [116]:
# Reset Index
df_z1.reset_index(drop=True, inplace=True)

### Zone 2&3

In [117]:
# Split Well Name into 3 columns: Part1, Part2 (first paren), Part3 (second paren)
def split_well_name_z23(val):
    s = str(val).strip() if pd.notna(val) else ""
    if not s:
        return "", "", ""
    # Pattern: Part1\n(Part2)\n (Part3)
    m = re.match(r"^([^\n]*)\n?\(([^)]*)\)\s*\n?\s*\(([^)]*)\)\s*$", s)
    if m:
        p1, p2, p3 = m.group(1).strip(), m.group(2).strip(), m.group(3).strip()
    else:
        # Try single paren: Part1\n(Part2)
        m2 = re.match(r"^([^\n]*)\n?\(([^)]*)\)\s*$", s)
        if m2:
            p1, p2, p3 = m2.group(1).strip(), m2.group(2).strip(), ""
        else:
            p1, p2, p3 = s, "", ""
    # Fallback: if Part 1 or Part 3 blank, copy from Part 2
    if not p1 and p2:
        p1 = p2
    if not p3 and p2:
        p3 = p2
    return p1, p2, p3

df_z23 = df[df["Zone"] == "Zone 2&3"].copy()
split_result = df_z23["Well Name"].apply(split_well_name_z23)
# Part 2 -> Well Name, Part 3 -> Well Name [2] (drop Part 1)
df_z23["Well Name"] = split_result.apply(lambda x: x[1])
df_z23["Well Name [2]"] = split_result.apply(lambda x: x[2])

# Sort by column Rig Name
df_z23 = df_z23.sort_values(by="Rig Name")

In [118]:
# Split Summary Report into 3 columns: Summary Report, Current Status, Next Plan
# Keywords: Laporan:, Status Pagi HH:MM:, Rencana:

def split_summary_report_z23(val):
    s = str(val).strip() if pd.notna(val) else ""
    if not s:
        return "", "", ""
    s = s.replace("_x000D_", "\n")  # Normalize Excel carriage return

    # Extract Laporan (Summary Report) - strip leading "-"
    summary = ""
    m1 = re.search(r"Laporan:\s*(.*?)(?=Status Pagi|Rencana:|$)", s, re.DOTALL)
    if m1:
        summary = m1.group(1).strip()
        if summary.startswith("-"):
            summary = summary[1:].strip()

    # Extract Status Pagi (Current Status) - flexible HH:MM or H:MM
    status = ""
    m2 = re.search(r"Status Pagi\s*\d{1,2}:\d{2}\s*:\s*(.*?)(?=Rencana:|$)", s, re.DOTALL)
    if m2:
        status = m2.group(1).strip()

    # Extract Rencana (Next Plan)
    plan = ""
    m3 = re.search(r"Rencana:\s*(.*)$", s, re.DOTALL)
    if m3:
        plan = m3.group(1).strip()

    return summary, status, plan

split_result = df_z23["Summary Report"].apply(split_summary_report_z23)
df_z23["Summary Report"] = split_result.apply(lambda x: x[0])
df_z23["Current Status"] = split_result.apply(lambda x: x[1])
df_z23["Next Plan"] = split_result.apply(lambda x: x[2])

### Zone 4

In [119]:
# Split Well Name: Part1 (Part2) -> Well Name, Well Name [2]
# If Well Name [2] is blank, copy from Well Name

def split_well_name_z4(val):
    s = str(val).strip() if pd.notna(val) else ""
    if not s:
        return "", ""
    m = re.match(r"^(.+?)\s*\(([^)]*)\)\s*$", s)
    if m:
        p1, p2 = m.group(1).strip(), m.group(2).strip()
        if not p2:
            p2 = p1
    else:
        p1, p2 = s, s
    return p1, p2

df_z4 = df[df["Zone"] == "Zone 4"].copy()
df_z4["Well Name"] = df_z4["Well Name"].str.replace("\u2060", "", regex=False)

split_result = df_z4["Well Name"].apply(split_well_name_z4)
df_z4["Well Name"] = split_result.apply(lambda x: x[0])
df_z4["Well Name [2]"] = split_result.apply(lambda x: x[1])
df_z4.reset_index(drop=True, inplace=True)

In [120]:
df_z4["Rig Name"] = df_z4["Rig Name"].str.replace("Rig", "", regex=False).str.strip()

# Replace known names, and normalize whitespace after '#' for 'PDSI' rigs
df_z4["Rig Name"] = (
    df_z4["Rig Name"]
    .replace({
        "Airlangga #55": "Airlangga-55",
        "PDSI ACS#21": "ACS-21",
        "#36.1/Skytop 650M": "PDSI #36.1/Skytop 650M"
    })
    .apply(lambda x: re.sub(r'(PDSI #)\s+', r'\1', x) if isinstance(x, str) and x.startswith('PDSI #') else x)
)

# Sort by column Rig Name
df_z4 = df_z4.sort_values(by="Rig Name")

In [121]:
# Split Summary Report into 3 columns: Summary Report, Current Status, Next Plan
# Keywords: Status Pagi, Plan:

def split_summary_report_z4(val):
    s = str(val).strip() if pd.notna(val) else ""
    if not s:
        return "", "", ""
    s = s.replace("_x000D_", "\n")  # Normalize Excel carriage return

    # Summary Report: content from start until Status Pagi or Plan:
    summary = ""
    m1 = re.search(r"^(.+?)(?=Status Pagi|Plan:|$)", s, re.DOTALL)
    if m1:
        summary = m1.group(1).strip()

    # Current Status: after Status Pagi (optional time) : until Plan:
    status = ""
    m2 = re.search(r"Status Pagi(?:\s*\d{1,2}:\d{2})?\s*:\s*(.*?)(?=Plan:|$)", s, re.DOTALL)
    if m2:
        status = m2.group(1).strip()

    # Next Plan: after Plan: until end
    plan = ""
    m3 = re.search(r"Plan:\s*(.*)$", s, re.DOTALL)
    if m3:
        plan = m3.group(1).strip()

    return summary, status, plan

split_result = df_z4["Summary Report"].apply(split_summary_report_z4)
df_z4["Summary Report"] = split_result.apply(lambda x: x[0])
df_z4["Current Status"] = split_result.apply(lambda x: x[1])
df_z4["Next Plan"] = split_result.apply(lambda x: x[2])

### Merge and display

In [122]:
# Merge Zone 1, Zone 2&3, and Zone 4 DataFrames
df_merged = pd.concat([df_z1, df_z23, df_z4], ignore_index=True)

In [ ]:
# Add date picker widget and display dataframe (for df_merged)
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import timedelta

# Date picker and navigation buttons
date_picker_merged = widgets.DatePicker(description='Pick a date:', disabled=False)
btn_decrement_merged = widgets.Button(description='◀ Previous Day', button_style='info')
btn_increment_merged = widgets.Button(description='Next Day ▶', button_style='info')

# Navigation button callbacks
def decrement_date_merged(btn):
    if date_picker_merged.value is not None:
        date_picker_merged.value = date_picker_merged.value - timedelta(days=1)

def increment_date_merged(btn):
    if date_picker_merged.value is not None:
        date_picker_merged.value = date_picker_merged.value + timedelta(days=1)

btn_decrement_merged.on_click(decrement_date_merged)
btn_increment_merged.on_click(increment_date_merged)
widget_box_merged = widgets.HBox([btn_decrement_merged, date_picker_merged, btn_increment_merged])

# Filtering and display function
def show_df_by_date_merged(change):
    clear_output(wait=True)
    display(widget_box_merged)
    selected_date = date_picker_merged.value
    if selected_date is not None:
        filtered_df = df_merged[df_merged["Report Date"] == selected_date]
    else:
        filtered_df = df_merged
    display(filtered_df.head(3))
    filtered_df.to_clipboard(header=False, index=False)

# Initial display
clear_output(wait=True)
display(widget_box_merged)
display(df_merged.head(3))
date_picker_merged.observe(show_df_by_date_merged, names='value')


,Flag,Region,Zone,APH,Rig Name,Well Name,Well Name [2],Well Type,Location,Spud Date,Release Date,Status,Status Code [1],Status Code [2],Summary Report,Current Status,Next Plan,Report Date,Operation Date
3,INC,Region 1,Zone 1,PEP,APS-752,RNT-DZ51,P-475,Development,Onshore,,,,,,Persiapan inspeksi kategori 3,,Rig Down 100%. Inspeksi Kategori 3.\nNote:\n- ...,2026-02-12,2026-02-11
17,INC,Region 1,Zone 2&3,PHR,ACS-009,Rig ACS-009 Activity at Yard,Rig ACS-009 Activity at Yard,Development,Onshore,,,,,,Yearly maintenance\n\nYM progress : 74.38% (ba...,,Continue Yearly maintenance,2026-02-12,2026-02-11
31,INC,Region 1,Zone 2&3,PHR,ACS-019,Duri 11519,5H-0305B,Development,Onshore,,,,,,"Drilling 11-3/4"", Circulate Hole Clean, POOH &...",,"Gravel Pack Jobs, Prepared for tubing pump run...",2026-02-12,2026-02-11
